**Title**

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
# !pip install transformers
!pip install datasets

In [13]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, BartForConditionalGeneration
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
# AutoModelForConditionalGeneration

BART
* Base model has no language head (output is embeddings)
* CausalLM is decoder-only but not all weights are pretrained
* ConditionalGeneration is the same as the base model but includes the language head

In [14]:
model_name = 'facebook/bart-base'
model_name = 'facebook/bart-large-cnn'
# model_name = 'gpt2-medium'
# model_name = 'facebook/bart-large'
# model = AutoModel.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
# print(model)
dir(model)
model.config
model.config.max_position_embeddings # maximum input sequence length

BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "L

In [43]:
input_text = r"Medical Specialty:\
General Medicine\
Sample Name: Abdominal Pain - Consult\
Description: The patient presented to the emergency room last evening with approximately 7- to 8-day history of abdominal pain which has been persistent.\
(Medical Transcription Sample Report)\
CHIEF COMPLAINT: Abdominal pain.\
HISTORY OF PRESENT ILLNESS: The patient is a 71-year-old female patient of Dr. X. The patient presented to the emergency room last evening with approximately 7- to 8-day history of abdominal pain which has been persistent. She was seen 3 to 4 days ago at ABC ER and underwent evaluation and discharged and had a CT scan at that time and she was told it was 'normal.' She was given oral antibiotics of Cipro and Flagyl. She has had no nausea and vomiting, but has had persistent associated anorexia. She is passing flatus, but had some obstipation symptoms with the last bowel movement two days ago. She denies any bright red blood per rectum and no history of recent melena. Her last colonoscopy was approximately 5 years ago with Dr. Y. She has had no definite fevers or chills and no history of jaundice. The patient denies any significant recent weight loss.\
PAST MEDICAL HISTORY: Significant for history of atrial fibrillation, under good control and now in normal sinus rhythm and on metoprolol and also on Premarin hormone replacement.\
PAST SURGICAL HISTORY: Significant for cholecystectomy, appendectomy, and hysterectomy. She has a long history of known grade 4 bladder prolapse and she has been seen in the past by Dr. Chip Winkel, I believe that he has not been re-consulted.\
ALLERGIES: SHE IS ALLERGIC OR SENSITIVE TO MACRODANTIN.\
SOCIAL HISTORY: She does not drink or smoke.\
REVIEW OF SYSTEMS: Otherwise negative for any recent febrile illnesses, chest pains or shortness of breath.\
PHYSICAL EXAMINATION:\
GENERAL: The patient is an elderly thin white female, very pleasant, in no acute distress.\
VITAL SIGNS: Her temperature is 98.8 and vital signs are all stable, within normal limits.\
HEENT: Head is grossly atraumatic and normocephalic. Sclerae are anicteric. The conjunctivae are non-injected.\
NECK: Supple.\
CHEST: Clear.\
HEART: Regular rate and rhythm.\
ABDOMEN: Generally nondistended and soft. She is focally tender in the left lower quadrant to deep palpation with a palpable fullness or mass and focally tender, but no rebound tenderness. There is no CVA or flank tenderness, although some very minimal left flank tenderness.\
PELVIC: Currently deferred, but has history of grade 4 urinary bladder prolapse.\
EXTREMITIES: Grossly and neurovascularly intact.\
LABORATORY VALUES: White blood cell count is 5.3, hemoglobin 12.8, and platelet count normal. Alkaline phosphatase elevated at 184. Liver function tests otherwise normal. Electrolytes normal. Glucose 134, BUN 4, and creatinine 0.7.\
DIAGNOSTIC STUDIES: EKG shows normal sinus rhythm.\
IMPRESSION AND PLAN: A 71-year-old female with greater than one-week history of abdominal pain now more localized to the left lower quadrant. Currently is a nonacute abdomen. The working diagnosis would be sigmoid diverticulitis. She does have a history in the distant past of sigmoid diverticulitis. I would recommend a repeat stat CT scan of the abdomen and pelvis and keep the patient nothing by mouth. The patient was seen 5 years ago by Dr. Y in Colorectal Surgery. We will consult her also for evaluation. The patient will need repeat colonoscopy in the near future and be kept nothing by mouth now empirically. The case was discussed with the patient's primary care physician, Dr. X. Again, currently there is no indication for acute surgical intervention on today's date, although the patient will need close observation and further diagnostic workup."

In [5]:
input_text = r"This time, Icarus had followed his dad’s instructions.\
He’d taxied along the runway and was about to take off when his earbuds had fallen out and he’d lost contact with air traffic control. His legs were almost exhausted.\
Then, a bit of luck. A jumbo jet had assisted his take-off, boosting him into the air. The pilot had gesticulated that Icarus should get off the nose cone but that was easier said than done, although finally, he’d freed himself and plummeted to earth.\
​Thankfully, his fall was broken by the softened wax and broken feathers.\
Back to the drawing board."

In [6]:
# input_text = "Once upon a time"
# input_text = 'Create a story from this prompt: once upon a time'
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids, max_length=1000)
print(tokenizer.decode(output[0], skip_special_tokens=True))
# print(tokenizer.decode(output[0], skip_special_tokens=False))

This time, Icarus had followed his dad’s instructions. His earbuds had fallen out and he’d lost contact with air traffic control. A jumbo jet had assisted his take-off, boosting him into the air. The pilot had gesticulated that Icarus should get off the nose cone but that was easier said than done.


In [15]:
# https://huggingface.co/datasets/har1/MTS_Dialogue-Clinical_Note?row=8
# All the data is in the 'train' split, but the documentation says 1201 training samples, 100 validation samples
ds = load_dataset("har1/MTS_Dialogue-Clinical_Note", streaming=False)

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['ID', 'section_header', 'section_text', 'dialogue'],
        num_rows: 1301
    })
})

In [9]:
ds['train'][0]['section_text']
# ds['train'][0]['dialogue']
# ds['train'][0]

'Symptoms: no fever, no chills, no cough, no congestion, no nausea, no vomiting, no chest pain, no chest pressure.\nDiagnosis: hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis, kidney stones\nHistory of Patient: 76-year-old white female, presents to the clinic today originally for hypertension and a med check, followed by Dr. Kumar, issues stable\nPlan of Action: N/A'

In [17]:
tokens = tokenizer.tokenize(ds['train'][0]['section_text'])
ids = tokenizer.convert_tokens_to_ids(tokens)
decoded_string = tokenizer.decode(ids)
print(decoded_string)

Symptoms: no fever, no chills, no cough, no congestion, no nausea, no vomiting, no chest pain, no chest pressure.
Diagnosis: hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis, kidney stones
History of Patient: 76-year-old white female, presents to the clinic today originally for hypertension and a med check, followed by Dr. Kumar, issues stable
Plan of Action: N/A


In [27]:
dir(tokenizer)
tokenizer

BartTokenizerFast(name_or_path='facebook/bart-large-cnn', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

In [11]:
tokenizer_out = tokenizer(ds['train'][0]['section_text'], return_tensors='pt')
tokenizer_out

{'input_ids': tensor([[    0, 46994, 47629,    35,   117, 11696,     6,   117,  1855,  5622,
             6,   117, 21768,     6,   117, 15744,     6,   117, 27214,     6,
           117, 23600,     6,   117,  7050,  2400,     6,   117,  7050,  1164,
             4, 50118, 29038, 11244, 13310,    35, 30960,     6, 30311,   139,
         16880, 37830,     6, 30311,  1517,   368, 13310,     6, 15671, 37994,
         36866,  1809,     6, 28349, 16448,   179, 10100,     6, 12855, 15109,
         50118, 38261,     9, 27690,    35,  5553,    12,   180,    12,   279,
          1104,  2182,     6,  6822,     7,     5,  8474,   452,  3249,    13,
         30960,     8,    10,  5679,  1649,     6,  1432,    30,   925,     4,
          6706,     6,   743,  4375, 50118, 35351,     9,  5828,    35,   234,
            73,   250,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [12]:
type(tokenizer_out['input_ids'])

torch.Tensor

In [20]:
# Class to handle tokenization and dataloading

class TextPreprocessor:
    def __init__(self, tokenizer, model, dataset, input_col, output_col=None, padding="longest", truncation=True, batch_size=8):
        """
        Initialize the class with a tokenizer, model, dataset, and various tokenization options.
        - input_col: The column name for the input sequences.
        - output_col: The column name for the output sequences (if applicable). If None, assume single sequence input.
        - padding: Padding strategy (e.g., "max_length", "longest", or a specific integer length).
        - truncation: Truncation strategy (boolean or "longest_first").
        """
        self.tokenizer = tokenizer
        self.model = model
        self.model_max_length = model.config.max_position_embeddings
        self.dataset = dataset
        self.input_col = input_col
        self.output_col = output_col
        self.padding = padding
        self.truncation = truncation
        self.batch_size = batch_size
        self.tokenized_dataset = None
        self.dataloader = None

    def tokenize_function(self, example):
        # Tokenize a pair of sequences (input-output pair, e.g. for summarization)
        if self.output_col:
            return self.tokenizer(
                example[self.input_col],
                text_target=example[self.output_col],   # target sequence
                padding=self.padding,
                truncation=self.truncation,
                max_length=self.model_max_length,
                return_tensors="pt"
            )

        # Tokenize a single sequence
        else:
            return self.tokenizer(
                example[self.input_col],
                padding=self.padding,
                truncation=self.truncation,
                max_length=self.model_max_length,
                return_tensors="pt"
            )

    def tokenize_dataset(self):
        """Applies tokenization to the dataset using the tokenize_function."""
        self.tokenized_dataset = self.dataset.map(self.tokenize_function, batched=True)

    def create_dataloader(self):
        """Creates a DataLoader from the tokenized dataset."""
        if self.tokenized_dataset is None:
            raise ValueError("Tokenized dataset is not available. Run tokenize_dataset() first.")

        # Convert to PyTorch DataLoader for batching
        self.dataloader = DataLoader(self.tokenized_dataset, batch_size=self.batch_size, shuffle=True)

    def get_dataloader(self):
        """Returns the DataLoader, creating it if necessary."""
        if self.dataloader is None:
            self.create_dataloader()
        return self.dataloader

In [21]:
preprocessor = TextPreprocessor(tokenizer, model, ds, 'dialogue', output_col='section_text')

preprocessor.tokenize_dataset()
preprocessor.create_dataloader()

Map:   0%|          | 0/1301 [00:00<?, ? examples/s]

In [22]:
preprocessor.tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'section_header', 'section_text', 'dialogue', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1301
    })
})

In [25]:
preprocessor.tokenized_dataset['train']['input_ids'][0]

[0,
 41152,
 35,
 653,
 3291,
 47,
 124,
 88,
 5,
 8474,
 452,
 6,
 2649,
 116,
 1437,
 50118,
 18276,
 4843,
 35,
 38,
 376,
 11,
 13,
 10,
 40453,
 9,
 127,
 1925,
 1164,
 6150,
 4,
 1437,
 50118,
 41152,
 35,
 85,
 1326,
 101,
 12521,
 6706,
 1432,
 62,
 19,
 47,
 94,
 86,
 2624,
 110,
 30960,
 6,
 30311,
 139,
 16880,
 37830,
 6,
 30311,
 1517,
 368,
 13310,
 6,
 15671,
 37994,
 36866,
 1809,
 6,
 28349,
 16448,
 179,
 10100,
 8,
 12855,
 15109,
 4,
 1437,
 6319,
 47,
 5324,
 143,
 1022,
 50,
 109,
 47,
 33,
 143,
 1379,
 2624,
 209,
 743,
 116,
 1437,
 1437,
 50118,
 18276,
 4843,
 35,
 440,
 4,
 1437,
 50118,
 41152,
 35,
 6319,
 47,
 56,
 143,
 11696,
 50,
 1855,
 5622,
 6,
 21768,
 6,
 15744,
 6,
 27214,
 6,
 23600,
 6,
 7050,
 2400,
 6,
 7050,
 1164,
 116,
 50118,
 18276,
 4843,
 35,
 440,
 4,
 1437,
 1437,
 50118,
 41152,
 35,
 2860,
 4,
 1578,
 6,
 13,
 84,
 2189,
 6,
 141,
 793,
 32,
 47,
 8,
 99,
 1015,
 109,
 47,
 3058,
 2512,
 25,
 116,
 50118,
 18276,
 4843,
 35,
 38,
 